In [2]:

import requests
import json
import pandas as pd
api_key = ""

Below we obtain the name of all news sources with a political focus based inthe united states. We then convert this json object to a dictionary and serach for sources through it. 

In [3]:
# Get all news outlets with a political focus within the US

url = "https://api.newscatcherapi.com/v2/sources"
querystring = {"lang":"en","countries":"US","topic":"politics"}
headers = {
    "x-api-key": api_key
    }
response = requests.request("GET", url, headers=headers, params=querystring)
print(response.text)

{"message":"Maximum sources displayed according to your plan is set to 1000","sources":["exbulletin.com","newsmax.com","politicalwire.com","dailycaller.com","realclearpolitics.com","republicworld.com","conservativereview.com","foxnews.com","cnn.com","thefederalist.com","clickondetroit.com","floridapolitics.com","crooksandliars.com","conservativehome.com","ijr.com","cbs.com","ksat.com","news24.com","palmerreport.com","amgreatness.com","kgou.org","politicaldog101.com","channel4000.com","kasu.org","nationalreview.com","wsiu.org","wuwm.com","wvasfm.org","wvia.org","americanthinker.com","apr.org","bignewsnetwork.com","heritage.org","kosu.org","lifenews.com","localnews8.com","whio.com","wsbradio.com","checkyourfact.com","click2houston.com","hotair.com","independentsentinel.com","krmg.com","lewrockwell.com","middletownpress.com","nationalmemo.com","wadk.com","wdbo.com","wokv.com","wpxi.com","boston25news.com","kclu.org","newrepublic.com","nhregister.com","pbs.org","thenation.com","theweek.com

In [4]:
# Convert query to python dictionary 
sources_json = json.loads(response.text)

In [20]:
# look up sources containing a given substring 
for source in sources_json["sources"]:
    print(source)
    #if "Glenn" in source: print(source)

exbulletin.com
newsmax.com
politicalwire.com
dailycaller.com
realclearpolitics.com
republicworld.com
conservativereview.com
foxnews.com
cnn.com
thefederalist.com
clickondetroit.com
floridapolitics.com
crooksandliars.com
conservativehome.com
ijr.com
cbs.com
ksat.com
news24.com
palmerreport.com
amgreatness.com
kgou.org
politicaldog101.com
channel4000.com
kasu.org
nationalreview.com
wsiu.org
wuwm.com
wvasfm.org
wvia.org
americanthinker.com
apr.org
bignewsnetwork.com
heritage.org
kosu.org
lifenews.com
localnews8.com
whio.com
wsbradio.com
checkyourfact.com
click2houston.com
hotair.com
independentsentinel.com
krmg.com
lewrockwell.com
middletownpress.com
nationalmemo.com
wadk.com
wdbo.com
wokv.com
wpxi.com
boston25news.com
kclu.org
newrepublic.com
nhregister.com
pbs.org
thenation.com
theweek.com
wonkette.com
balloon-juice.com
bgdailynews.com
bipartisanreport.com
commondreams.org
kfab.com
mediaite.com
news4jax.com
redstate.com
thegatewaypundit.com
thehour.com
wjtv.com
wsoctv.com
yahoo.com
19th

Below is an example of a query serach for articles from CNN for a specific time frame provided. The query result provides us with the first fifty hits as well as information about additional articles. 

In [28]:
search_url = "https://api.newscatcherapi.com/v2/search"

search_querystring = {"q":"*","lang":"en","countries":"US", "topic":"politics", 
            "sources":["cnn.com"],"page":"1", "from":"2022/04/01", "to":"2022/07/01"}

headers = {
    "x-api-key": api_key
    }

response = requests.request("GET", search_url, headers=headers, params=search_querystring)

print(response.text)


{"status":"ok","total_hits":1066,"page":1,"total_pages":22,"page_size":50,"articles":[{"title":"Thomas says government institutions shouldn't be 'bullied' following leak of draft opinion on abortion","author":"Ariane de Vogue, CNN Supreme Court Reporter","published_date":"2022-05-06 23:40:01","published_date_precision":"full","link":"https://www.cnn.com/2022/05/06/politics/clarence-thomas-stare-decisis-roe-v-wade-leak/index.html","clean_url":"cnn.com","excerpt":"Justice Clarence Thomas, following the leak of a draft majority opinion overturning Roe v. Wade, said Friday that government institutions shouldn't be \"bullied\" into delivering what some see as the\u2026","summary":"Atlanta (CNN)Justice Clarence Thomas, following the leak of a draft majority opinion overturning Roe v. Wade, said Friday that government institutions shouldn't be \"bullied\" into delivering what some see as the preferred outcome. \"The events of earlier this week is one symptom of that,\" Thomas said. He added t

the getArticles function performs a query serach for a specified news outlet, within a gspecified time frame 

In [10]:
def getArticles(source, start_date, end_date, page):
    search_url = "https://api.newscatcherapi.com/v2/search"
    search_querystring = {"q":"*","lang":"en","countries":"US", "topic":"politics", 
                "sources":[source],"page":page, "from":start_date, "to":end_date}
    headers = {
        "x-api-key": api_key
        }
    response = requests.request("GET", search_url, headers=headers, params=search_querystring)
    articles_dic = json.loads(response.text)
    return articles_dic
    


This is the query search loop initially used. It proved to be faulty as it queries too many articles and didnt check whether an article text was available before appending it to the data frame. 

In [ ]:

# sources = ['newsmax.com','vdare.com', 'newsnationnow.com', 'politicawire.com', 'palmerreport.com', 'bipartisanreport.com']
sources = ['pbs.org']
years = ["2021", "2022", "2023"]


for source in sources:
    df = pd.DataFrame()
    for year in years:
        start_date = year + '/01/01'
        end_date = year + '/12/31'
        pages = getArticles(source,start_date, end_date,"1")["total_pages"]
        for page in range(1, pages):
            articles = getArticles(source,start_date, end_date,str(page))["articles"]
            temp_df = pd.DataFrame(articles)
            df = df.append(temp_df, ignore_index=True)

    file_name = source[:len(source)-3] + 'csv'
    clean_df = df[['title', 'published_date', 'clean_url', 'summary']]
    clean_df.to_csv(file_name)
    

This is the query loop ultimatley used. It performs much more predictably by checking for the presence of article text and limiting the number of articles than can be queried within a given time period. 

In [39]:
# Define all news sources being queried
sources = ['newsmax.com','vdare.com', 'newsnationnow.com', 'palmerreport.com', 'bipartisanreport.com', 'pbs.org', 'newsnationnow.com', 'apnews.com']
# Define all time intervals foor querying 
dates = [("2021/04/01", "2021/06/30"), ("2021/07/01", "2021/09/30"), 
        ("2021/10/01", "2021/12/30"), ("2022/01/01", "2022/03/30"), 
        ("2022/04/01", "2022/06/30"), ("2022/07/01", "2022/09/30"), 
        ("2022/10/01", "2022/12/30"), ("2023/01/01", "2023/03/30"),
        ("2023/04/01", "2023/06/30"), ("2023/07/01", "2023/09/30")]
# for each news outlet 
for source in sources:
    df = pd.DataFrame()
    # For each interval
    for interval in dates:
        # Set start and end dae and pull articles from every third query page 
        start_date = interval[0]
        end_date = interval[1]
        pages = getArticles(source,start_date, end_date,"1")["total_pages"]
        step = pages // 3
        for page in range(1, pages, step):
            articles = getArticles(source,start_date, end_date,str(page))["articles"]
            temp_df = pd.DataFrame(articles)
            temp_df = temp_df[temp_df['summary'] != '']
            df = df.append(temp_df, ignore_index=True)
            # set cap on  articles per time frame 
            if len(df) > 3000:
                    break
    file_name = source[:len(source)-3] + 'csv'
    clean_df = df[['title', 'published_date', 'clean_url', 'summary']]
    clean_df.to_csv(file_name)

/var/folders/r_/wwpxjnln7vx4bw2q6dxn_svm0000gn/T/ipykernel_61511/1361145821.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
/var/folders/r_/wwpxjnln7vx4bw2q6dxn_svm0000gn/T/ipykernel_61511/1361145821.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
/var/folders/r_/wwpxjnln7vx4bw2q6dxn_svm0000gn/T/ipykernel_61511/1361145821.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
/var/folders/r_/wwpxjnln7vx4bw2q6dxn_svm0000gn/T/ipykernel_61511/1361145821.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
 